Вводим чанекр. На входе документ с запросами website_updated.xlsx


In [28]:
#!/usr/bin/env python3
# src/preprocessing/chunker.py
"""
Чанкинг websites_updated.xlsx или websites_updated.csv → chunks.csv и chunks.jsonl
Вход: data/raw/websites_updated.xlsx или .csv
Выход: data/processed/chunks.csv, data/processed/chunks.jsonl

Особенности:
- Работает с .xlsx (через openpyxl) и .csv (через pandas)
- Контекстный префикс: kind, title — отдельно, text — только текст
- Глубокая очистка: ?oirutpspid=, tel., footer-фразы, JSON-фрагменты (из конфига)
- Семантический чанкирование: через Sentence Transformers + Similarity (быстрее, чем кластеризация)
- Фильтрация коротких чанков (<100 симв.)
- Дедупликация по MD5(text)
- НОВОЕ: метрики добавляются в конец CSV и JSONL
- НОВОЕ: логгирование в файл chunker.log
- НОВОЕ: оба файла (CSV и JSONL) генерируются всегда
- НОВОЕ: автоматический выбор CPU/GPU
- НОВОЕ: исправлена ошибка os.makedirs при пустом dirname
"""

import argparse
import os
import re
import hashlib
import json
import logging
from tqdm import tqdm
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch


# --- Автоматический выбор устройства ---
def choose_device():
    if torch.cuda.is_available():
        # Проверим VRAM (в ГБ)
        total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        # Если VRAM < 4 ГБ — не используем GPU (например, GT 1030)
        if total_memory < 4:
            print(f"⚠️ GPU доступна, но VRAM = {total_memory:.1f} ГБ — используем CPU")
            return 'cpu'
        else:
            print(f"✅ Используем GPU: {torch.cuda.get_device_name(0)}")
            return 'cuda'
    else:
        print("⚠️ GPU недоступна — используем CPU")
        return 'cpu'


# --- Настройка логгирования ---
def setup_logger(output_dir: str):
    log_path = os.path.join(output_dir, "chunker.log")
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.FileHandler(log_path, mode="w", encoding="utf-8"),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger("chunker")


# --- Загрузка NOISE_PATTERNS из конфига (в той же папке) ---
def load_noise_patterns(config_path: str = "noise_patterns.json"):
    default_patterns = [
        r'\?oirutpspid=[^&\s]*',
        r'\?oirutpspsc=[^&\s]*',
        r'\?oirutpspjs=[^&\s]*',
        r'#\S*',
        r'tel\.', r'Tel\.', r'тел\.', r'Тел\.', r'тел:', r'Тел:',
        r'©\s*Альфа-Банк',
        r'Пользуясь сайтом',
        r'соглашаетесь с политикой конфиденциальности',
        r'Контакты', r'Поддержка', r'Карта сайта',
        r'Москва', r'Россия',
        r'199\d–202\d',
        r'https?://[^\s]+\.(?:png|jpg|jpeg|gif|pdf)',
        r'\{[^{}]*\}', # JSON-фрагменты
    ]

    if os.path.exists(config_path):
        with open(config_path, "r", encoding="utf-8") as f:
            try:
                config = json.load(f)
                return config.get("noise_patterns", default_patterns)
            except Exception:
                print(f"⚠️ Ошибка загрузки конфига {config_path}, используем стандартные паттерны")
                return default_patterns
    else:
        print(f"⚠️ Конфиг {config_path} не найден, используем стандартные паттерны")
        return default_patterns


NOISE_PATTERNS = load_noise_patterns()


def clean_text(s: str) -> str:
    """Удаление HTML и шума"""
    if not isinstance(s, str):
        return ""
    for pattern in NOISE_PATTERNS:
        s = re.sub(pattern, '', s, flags=re.IGNORECASE)
    s = re.sub(r'<script.*?</script>', ' ', s, flags=re.IGNORECASE | re.DOTALL)
    s = re.sub(r'<style.*?</style>', ' ', s, flags=re.IGNORECASE | re.DOTALL)
    s = re.sub(r'<[^>]+>', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    s = re.sub(r'([.,;!?])\1+', r'\1', s)
    return s.strip()


def semantic_chunk_by_similarity(text: str, model, max_chunk_len: int = 400, threshold: float = 0.5):
    """
    Семантическое чанкирование через сравнение эмбеддингов соседних предложений
    """
    sentences = [s.strip() for s in re.split(r'\n\s*\n|\. ', text) if s.strip()]
    if not sentences:
        return []

    if len(sentences) < 2:
        # Просто разбиваем по длине
        chunks = []
        current_chunk = ""
        for sent in sentences:
            if len(current_chunk) + len(sent) < max_chunk_len:
                current_chunk += " " + sent
            else:
                if current_chunk.strip():
                    chunks.append(current_chunk.strip())
                current_chunk = sent
        if current_chunk.strip():
            chunks.append(current_chunk.strip())
        return [c for c in chunks if len(c) >= 100]

    embeddings = model.encode(sentences)

    chunks = []
    current_chunk = sentences[0]
    current_emb = embeddings[0].reshape(1, -1)

    for i in range(1, len(sentences)):
        next_emb = embeddings[i].reshape(1, -1)
        similarity = cosine_similarity(current_emb, next_emb)[0][0]

        if similarity > threshold and len(current_chunk) + len(sentences[i]) < max_chunk_len:
            current_chunk += " " + sentences[i]
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentences[i]
            current_emb = next_emb

    if current_chunk:
        chunks.append(current_chunk.strip())

    return [c for c in chunks if len(c) >= 100]


def hash_text(text: str) -> str:
    """Хэширование текста для дедупликации"""
    return hashlib.md5(text.encode('utf-8', errors='ignore')).hexdigest()


def main(args):
    # Исправленная строка: проверяем, есть ли путь в output
    output_dir = os.path.dirname(args.output)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    logger = setup_logger(os.path.dirname(args.output))

    logger.info(f"Чтение {args.input}...")

    # Определяем расширение файла
    if args.input.lower().endswith('.xlsx'):
        try:
            df = pd.read_excel(args.input, engine="openpyxl")
        except ImportError:
            raise ImportError("Для чтения .xlsx нужен пакет 'openpyxl'. Установите его: pip install openpyxl")
    elif args.input.lower().endswith('.csv'):
        df = pd.read_csv(args.input)
    else:
        raise ValueError(f"Файл {args.input} не является .csv или .xlsx")

    required_cols = {"web_id", "url", "kind", "title", "text"}
    if not required_cols.issubset(set(df.columns)):
        raise SystemExit(f"Ожидались колонки: {required_cols}. Есть: {set(df.columns)}")

    logger.info(f"Обрабатываем {len(df)} документов")

    # Выбираем устройство автоматически
    device = choose_device()

    # Загружаем модель один раз на выбранное устройство
    logger.info(f"Загрузка модели {args.model_name} на {device}...")
    model = SentenceTransformer(args.model_name, device=device)

    rows_csv = []
    rows_jsonl = []

    web_ids = set()
    seen_chunk_ids = set()

    for _, r in tqdm(df.iterrows(), total=len(df), desc="Чанкинг"):
        # Проверки
        web_id_raw = r.get("web_id")
        if pd.isna(web_id_raw):
            logger.warning(f"Пропуск строки: web_id пустой: {r.name}")
            continue
        try:
            web_id = int(web_id_raw)
        except (ValueError, TypeError):
            logger.warning(f"Пропуск строки: web_id не конвертируется в int: {web_id_raw}")
            continue

        title = r.get("title")
        url = r.get("url")
        kind = r.get("kind")
        raw_text = r.get("text")

        if pd.isna(title) or pd.isna(kind) or pd.isna(raw_text):
            logger.warning(f"Пропуск строки: один из обязательных полей пуст: {r.name}")
            continue

        title = str(title).strip()
        url = str(url).strip() if pd.notna(url) else ""
        kind = str(kind).strip()
        raw_text = str(raw_text).strip()

        if len(raw_text) < 100:
            logger.info(f"Пропуск строки: текст слишком короткий: {r.name}")
            continue

        # Очищаем и чанкуем
        clean_full_text = clean_text(raw_text)
        chunks = semantic_chunk_by_similarity(
            clean_full_text,
            model,
            max_chunk_len=args.max_chunk_len,
            threshold=args.threshold
        )

        for i, chunk_text in enumerate(chunks):
            chunk_id = f"{web_id}__{i}"

            if chunk_id in seen_chunk_ids:
                logger.warning(f"Пропуск дубликата chunk_id: {chunk_id}")
                continue
            seen_chunk_ids.add(chunk_id)

            # для CSV: text — только текст, без kind и title
            rows_csv.append({
                "web_id": web_id,
                "chunk_id": chunk_id,
                "title": title,
                "url": url,
                "kind": kind,
                "text": chunk_text
            })
            # для JSONL: тоже самое
            rows_jsonl.append({
                "chunk_id": chunk_id,
                "text": chunk_text,
                "url": url,
                "title": title,
                "web_id": web_id,
                "kind": kind
            })

        web_ids.add(web_id)

    # Дедупликация по text
    seen_hashes = set()
    unique_csv = []
    unique_jsonl = []
    for csv_row, jsonl_row in zip(rows_csv, rows_jsonl):
        h = hash_text(csv_row["text"])
        if h in seen_hashes:
            continue
        seen_hashes.add(h)
        unique_csv.append(csv_row)
        unique_jsonl.append(jsonl_row)

    logger.info(f"Удалено дубликатов: {len(rows_csv) - len(unique_csv)}")

    # --- СТАТИСТИКА ---
    chunks_count = len(unique_csv)
    total_len = sum(len(row["text"]) for row in unique_csv)
    avg_len = total_len / chunks_count if chunks_count > 0 else 0
    uniq_web_ids = len(web_ids)

    logger.info(f"📊 Статистика:")
    logger.info(f"   - Чанков: {chunks_count}")
    logger.info(f"   - Средняя длина: {avg_len:.2f}")
    logger.info(f"   - Уникальных web_id: {uniq_web_ids}")

    # --- СОХРАНЕНИЕ CSV ---
    out_df = pd.DataFrame(unique_csv, columns=["web_id", "chunk_id", "title", "url", "kind", "text"])
    out_df.to_csv(args.output, index=False)
    logger.info(f"chunks.csv: {len(out_df)} чанков")

    # --- ДОБАВЛЕНИЕ СТАТИСТИКИ В КОНЕЦ CSV ---
    with open(args.output, "a", encoding="utf-8") as f:
        f.write(f"\n#chunks_count,{chunks_count}\n")
        f.write(f"#avg_len,{avg_len:.2f}\n")
        f.write(f"#uniq_web_ids,{uniq_web_ids}\n")
    logger.info(f"✅ Метрики добавлены в конец {args.output}")

    # --- СОХРАНЕНИЕ JSONL ---
    jsonl_path = args.output.replace(".csv", ".jsonl")
    with open(jsonl_path, "w", encoding="utf-8") as f:
        for item in unique_jsonl:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")
        # --- ДОБАВЛЕНИЕ СТАТИСТИКИ В КОНЕЦ JSONL ---
        stats = {
            "chunks_count": chunks_count,
            "avg_len": round(avg_len, 2),
            "uniq_web_ids": uniq_web_ids
        }
        f.write(json.dumps(stats, ensure_ascii=False) + "\n")
    logger.info(f"chunks.jsonl: {len(unique_jsonl)} чанков → для Generator")
    logger.info(f"✅ Метрики добавлены в конец {jsonl_path}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Чанкинг websites_updated.xlsx или .csv → chunks.csv + chunks.jsonl")
    parser.add_argument('-f')
    parser.add_argument("--input", default="websites_updated.xlsx")
    parser.add_argument("--output", default="chunks.csv")
    parser.add_argument("--model_name", default="all-MiniLM-L6-v2", help="Модель для эмбеддингов (меньше/быстрее)")
    parser.add_argument("--max_chunk_len", type=int, default=400, help="Максимальная длина чанка (в символах)")
    parser.add_argument("--threshold", type=float, default=0.5, help="Порог схожести для объединения чанков")
    args = parser.parse_args()
    main(args)

⚠️ GPU недоступна — используем CPU


Чанкинг: 100%|██████████| 1938/1938 [1:22:00<00:00,  2.54s/it]


После того как созданы санки, на их основе строится векторная база через ретривер

In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 51.5 MB/s eta 0:00:00


In [29]:
import os
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import logging
import pandas as pd

def read_column_to_list(excel_file, column=0, sheet_name=0, header=0, skip_rows=0):
    try:
        df = pd.read_csv(excel_file,
                          header=header,
                          skiprows=skip_rows)

        if isinstance(column, int):
            selected_column = df.iloc[:, column].tolist()
        elif isinstance(column, str):
            selected_column = df[column].tolist()
        else:
            raise ValueError("Параметр 'column' должен быть int (индекс) или str (имя столбца)")

        return selected_column

    except FileNotFoundError:
        print(f"Ошибка: Файл '{excel_file}' не найден")
        return []
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
        return []

if __name__ == "__main__":
    data_column_list = read_column_to_list("chunks.csv", column=1)
    print(f"Получено {len(data_column_list)} элементов:")
    for i, item in enumerate(data_column_list[:2], 1):
        print(f"{i}: {item}")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class VectorIndexBuilder:
    def __init__(self, model_name='all-MiniLM-L6-v2', index_type='IVF'):
        self.model = SentenceTransformer(model_name)
        self.index_type = index_type
        self.dimension = self.model.get_sentence_embedding_dimension()

    def create_index(self, dimension):
        """Создание FAISS индекса в зависимости от типа"""
        if self.index_type == 'Flat':
            # Точный поиск
            return faiss.IndexFlatIP(dimension)
        elif self.index_type == 'IVF':
            # Приближенный поиск с кластеризацией
            nlist = 100  # количество кластеров
            quantizer = faiss.IndexFlatIP(dimension)
            index = faiss.IndexIVFFlat(quantizer, dimension, nlist)
            return index
        elif self.index_type == 'HNSW':
            # Графовый метод
            index = faiss.IndexHNSWFlat(dimension, 32)
            index.hnsw.efConstruction = 200
            return index
        else:
            raise ValueError(f"Unknown index type: {self.index_type}")

    def build_index(self, texts, output_dir="indexes", index_name="index"):
        """Построение полного индекса"""
        os.makedirs(output_dir, exist_ok=True)

        logger.info(f"Creating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True, batch_size=32)
        embeddings = embeddings.astype(np.float32)

        # Нормализация для косинусного сходства
        faiss.normalize_L2(embeddings)

        logger.info("Creating FAISS index...")
        index = self.create_index(self.dimension)

        if self.index_type == 'IVF':
            # Требуется обучение для IVF
            logger.info("Training IVF index...")
            index.train(embeddings)

        logger.info("Adding embeddings to index...")
        index.add(embeddings)

        # Сохранение метаданных
        metadata = {
            'texts': texts,
            'embedding_shape': embeddings.shape,
            'index_type': self.index_type,
            'model_name': 'all-MiniLM-L6-v2',
            'dimension': self.dimension
        }

        # Сохранение всех компонентов
        index_path = os.path.join(output_dir, f"{index_name}.faiss")
        meta_path = os.path.join(output_dir, f"{index_name}_meta.json")
        embeddings_path = os.path.join(output_dir, f"{index_name}_embeddings.npy")

        faiss.write_index(index, index_path)
        with open(meta_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        np.save(embeddings_path, embeddings)

        logger.info(f"Index saved to {index_path}")
        logger.info(f"Metadata saved to {meta_path}")
        logger.info(f"Embeddings saved to {embeddings_path}")

        return index, metadata, embeddings


if __name__ == "__main__":
    # Загрузка данных
    texts = read_column_to_list("chunks.csv", column=1)

    # Построение индекса
    builder = VectorIndexBuilder(index_type='IVF')
    index, metadata, embeddings = builder.build_index(texts)

    print("Index built successfully!")
    print(f"Index size: {index.ntotal} vectors")
    print(f"Embedding dimension: {metadata['dimension']}")

Получено 18110 элементов:
1: 1__0
2: 2__0


Batches:   0%|          | 0/566 [00:00<?, ?it/s]

Index built successfully!
Index size: 18110 vectors
Embedding dimension: 384


In [30]:
query = 'Номер счета'

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:

def embed(text):
    return model.encode([text])

import json
with open('indexes/index_meta.json', 'r') as f:
  metadata = json.load(f)

index = faiss.read_index("indexes/index.faiss")
query_emb = embed(query)
D, I = index.search(query_emb,k =200)
#results = [metadata[i] for i in I[0]]

#print(D)
print(I)


[[15190 15192 15189 15197 15193 15188 15196 15191 14930 15187 15199 15198
  14932 15194 14929 14933 15007 15041 15034 14931 15009 15047 15195 15035
  15129 15010 15032 15006 14934 15031 15131 15008 15040 14936 15048 15132
  15128 15042 14935 15137 15043 15212 15138 15033 15130 15140 15046 15067
  15139 15134 15036 15143 15133 15044 15045 15141 15049 15069 15144 15136
  15070 15039 15037 15066 15079 15081 15099 15090 15142 15075 15135 15076
  15083 15068 15103 15096 15102 15148 15146 15078 15089 15101 15077 15153
  15084 15071 15100 15104 15145 15072 15003 15105 15002 14342 15159 15074
  15005 15152 15097 15080 15051 15053 15082 15092 15155 15073 15085 15054
  15154 15091 15086 15093 15098 15147 15095 15059 15149 15158 15156 15061
  15052 15004 15151 15157 15060 15094 15062 15050 15160 15087 14701 15088
  15055 15058 15161 15065 15063 15064 15056 14939 14702 15057 14938 14941
  14940 14942    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1
     -1    -1    -1    -1    -1    -1 

In [32]:
df = pd.read_csv('chunks.csv').set_index('chunk_id')
print(df.head())

         web_id                                              title  \
chunk_id                                                             
1__0          1  Альфа-Банк - кредитные и дебетовые карты, кред...   
2__0          2                      А-Клуб. Деньги имеют значение   
2__1          2                      А-Клуб. Деньги имеют значение   
2__2          2                      А-Клуб. Деньги имеют значение   
2__3          2                      А-Клуб. Деньги имеют значение   

                                  url  kind  \
chunk_id                                      
1__0             https://alfabank.ru/  html   
2__0      https://alfabank.ru/a-club/  html   
2__1      https://alfabank.ru/a-club/  html   
2__2      https://alfabank.ru/a-club/  html   
2__3      https://alfabank.ru/a-club/  html   

                                                       text  
chunk_id                                                     
1__0      Персональные условия вы сможете узнать после 

In [33]:
chunk_ids = I[0]
web_ids = {int(df.iloc[i,0]) for i in chunk_ids if df.iloc[i,0] != '#uniq_web_ids'}


print(web_ids)
#

{1728, 1729, 1730, 1731, 1733, 1734, 1735, 1738, 1739, 1745, 1683, 1716, 1621, 1719, 1722, 1724, 1727}


In [34]:
docs = [df.iloc[i,4]  for i in I[0]]

Re-rank

In [25]:
from sentence_transformers import CrossEncoder

# Модель для re-ranking
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


In [35]:
web_doc = pd.read_csv('chunks.csv')

In [36]:
def rank(query, webs):
  docs = [web_doc.iloc[i,5] for i in webs]

  # Пары (запрос, документ)
  pairs = [(query, doc) for doc in docs]

  # Предсказание релевантности
  scores = reranker.predict(pairs)

  # Сортировка
  ranked = sorted(zip(scores, docs), reverse=True)

  # Печать топ-5
  print("Топ-5 документов после re-ranking:\n")
  y = []
  for i, (score, doc) in enumerate(ranked[:5], 1):
      #print(f"{i}. ({score:.4f}) {doc}")
      y.append(doc)
  return y

In [37]:
rank(query,web_ids)

Топ-5 документов после re-ranking:



['Например, чтобы получить одобрение в Альфа‑Банке, клиент должен: иметь гражданство РФ быть старше 21 года иметь непрерывный стаж работы не менее трёх месяцев В программе целевого автокредитования, кроме возраста, стажа и гражданства, важен уровень постоянного дохода: минимум — 10 000 рублей в месяц после удержания налогов',
 'Такие продавцы стремятся заключить сделку побыстрее, поэтому не обращают внимание на техническое состояние авто, не проверяют документы и занижают цены, чтобы избавиться от проблемной машины Для безопасности при выборе салона изучайте репутацию компании, отзывы, срок присутствия на вторичном рынке',
 'Проблема скудного выбора возникает из-за ограниченного числа брендов, с которыми работает дилерский центр, и корпоративной политики Дилеры не продают машины со значительными повреждениями или требующие ремонта А многие компании не берутся за продажу подержанных авто экономкласса, потому что такие сделки для них нерентабельны',
 'Технический специалист проведёт неза